<a href="https://colab.research.google.com/github/Nativeatom/RefSensitivity/blob/main/Referral_Expression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
proj_path = "/content/Referal_expr"

In [1]:
import os
from os.path import join
import json
import itertools
import random
import pdb

In [25]:
import torch
torch.__version__

'1.12.1+cu113'

In [ ]:
!nvcc -V

In [ ]:
!pip uninstall torch
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!bash /content/Referal_expr/setup.sh

In [ ]:
!bash /content/Referal_expr/mscoco_setup.sh

In [ ]:
import sys
sys.path.append("/content/GLIP")
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.engine.predictor_glip import GLIPDemo

In [ ]:
config_file = "GLIP/configs/pretrain/glip_Swin_T_O365_GoldG.yaml"
weight_file = "GLIP/MODEL/glip_tiny_model_o365_goldg_cc_sbu.pth"

cfg.local_rank = 0
cfg.num_gpus = 1
cfg.merge_from_file(config_file)
cfg.merge_from_list(["MODEL.WEIGHT", weight_file])
cfg.merge_from_list(["MODEL.DEVICE", "cuda"])

glip_demo = GLIPDemo(
    cfg,
    min_image_size=800,
    confidence_threshold=0.7,
    show_mask_heatmaps=False
)

NameError: ignored

In [2]:
def load(url, local=False):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    if local:
      pil_image = Image.open(url).convert("RGB")
    else:
      response = requests.get(url)
      pil_image = Image.open(BytesIO(response.content)).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def imshow(img, caption):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")
    plt.figtext(0.5, 0.09, caption, wrap=True, horizontalalignment='center', fontsize=20)

def edge2bbox(edge):
  x, y, w, h = edge
  return x, y, x+w, y+h

In [3]:
import zipfile
import io
from io import BytesIO
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pylab as pylab

from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import time
import pdb
import re

import torchvision.ops.boxes as bops
pylab.rcParams['figure.figsize'] = 20, 12

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
save_path = "/content/drive/MyDrive/Referal_expr/data/CLEVR"

In [ ]:
# https://www.geeksforgeeks.org/download-anything-to-google-drive-using-google-colab/
file_url = "https://www.cs.jhu.edu/~cxliu/data/clevr_ref+_1.0.zip"
    
r = requests.get(file_url, stream = True) 

with open(join(save_path, "clevr_ref+_1.0.zip"), "wb") as file: 
    for block in r.iter_content(chunk_size = 1024):
         if block: 
             file.write(block) 

In [ ]:
# Download and load the validation split of COCO-2017
dataset = foz.load_zoo_dataset("coco-2017", split="validation")

save_path = "/content/gdrive/MyDrive/Profile/Projects/NLP/Proposition"
all_data_info = {}

for file in os.listdir(save_path):
  if file.endswith(".json"):
    with open(join(save_path, file), "r") as fp:
      data = [json.loads(x) for x in fp.readlines()]
      all_data_info[file.replace(".json", "")] = data

list(all_data_info.keys())

In [ ]:
import time
for i in range(300):
  print(i)

  time.sleep(60)

In [ ]:
! gdown https://drive.google.com/uc?id=1UVLw5dKpgSGCOt2sha1d1Ezgpvlhf3G6

In [5]:
save_path = "/content/drive/MyDrive/Profile/Projects/NLP/Proposition/data"

refexps = 'clevr_ref+_1.0/refexps/clevr_ref+_train_refexps.json'

scene_file = 'clevr_ref+_1.0/scenes/clevr_ref+_val_scenes.json'

archive = zipfile.ZipFile(join(save_path, "clevr_ref+_1.0.zip"), 'r')

refexp_data = archive.read(scene_file)

refexp_data_str = refexp_data.decode('utf8')

refexp_data_json = json.loads(refexp_data_str)

# with open(join(save_path, "clevr_ref+_train_refexps.json"), "w") as fp:
#   json.dump(refexp_data_json, fp)

# with open(join(save_path, "clevr_ref+_train_scenes.json"), "w") as fp:
#   json.dump(refexp_data_json, fp)

In [ ]:
idx = 0
scene_info = refexp_data_json['scenes'][idx]
img_name = scene_info['image_filename']
sample_file = 'clevr_ref+_1.0/images/val/{}'.format(img_name)
imgdata = archive.read(sample_file)
print(img_name)

CLEVR_val_000000.png


In [ ]:
# create a pygame-compatible file-like object from the bytes
print(img_name)
# load the video scene
# img = pygame.image.load(bytes_io)

for obj in scene_info['objects']:
  bbox = scene_info['obj_bbox'][str(obj['idx'])]
  other_bboxes = [v for k, v in scene_info['obj_bbox'].items() if k != str(obj['idx'])]

  # bounding boxes that is in front of the target bbox
  other_bboxes_front = [edge2bbox(oth_bbox) for oth_bbox in other_bboxes if oth_bbox[1] < bbox[1]]
  max_iou = 0
  if len(other_bboxes_front):
    ious = bops.box_iou(torch.FloatTensor(other_bboxes_front), torch.FloatTensor(edge2bbox(bbox)).unsqueeze(0).repeat(len(other_bboxes_front), 1)).mean(1)
    max_iou = torch.max(ious).item()

  # read image
  # cv_img = cv2.imread(img_name)
  x, y, w, h = bbox

  bytes_io = io.BytesIO(imgdata)

  # https://stackoverflow.com/questions/64099834/how-can-i-display-an-image-in-python-thats-represented-as-an-array-of-bytes-wit
  with bytes_io:
      img = mpimg.imread(bytes_io, format='jpeg')

  cv2.rectangle(img, (int(x), int(y)), (int(x+w), int(y+h)), (255, 0, 0), 2)
  cv2_imshow(img)
  # plt.imshow(img)
  # plt.show()
  print("\n")

CLEVR_val_000299.png


NameError: ignored

In [ ]:
# https://github.com/facebookresearch/clevr-dataset-gen/blob/f0ce2c81750bfae09b5bf94d009f42e055f2cb3a/image_generation/render_images.py#L290
scene_info['directions']

{'right': [0.6563112735748291, 0.7544902563095093, -0.0],
 'below': [-0.0, -0.0, -1.0],
 'behind': [-0.754490315914154, 0.6563112735748291, 0.0],
 'above': [0.0, 0.0, 1.0],
 'front': [0.754490315914154, -0.6563112735748291, -0.0],
 'left': [-0.6563112735748291, -0.7544902563095093, 0.0]}

In [6]:
# max rotation to mantain the object facing the viewer
max_face_rotation_degree = 30

attrs_placeholder = {
    "rotation":  "<R>",
    "shape": "<S>",
    "size": "<Z>",
    "material": "<M>"
}

def color_group(x):
  return x['color']

def material_group(x):
  return x['material']

def shape_group(x):
  return x['shape']

def size_group(x):
  return x['size']

def rotation_group(x):
  # binary split of rotation
  # 90 degree rotation is equivalent to the 270 degree rotation given the shapes.
  # Not distinguishable for cubes.
  x = x["rotation"] % 90
  return x < max_face_rotation_degree

def collect_ambiguous_objects(scene, properties=['color', 'material', 'shape', 'size', 'rotation']):

  group_funcs = {
      "color": color_group,
      "material": material_group,
      "shape": shape_group,
      "size": size_group,
      "rotation": rotation_group
  }

  ambgs_obj_by_property = {}

  objects = scene['objects']

  for prprty in properties:
    if prprty == "rotation":
      # https://github.com/facebookresearch/clevr-dataset-gen/blob/f0ce2c81750bfae09b5bf94d009f42e055f2cb3a/image_generation/render_images.py#L411
      eligible_objects = [obj for obj in objects if obj['shape'] in ['cube']]
      ambgs_obj_by_property[prprty] = {"is_ambiguous": False, "attrs": None}
      
      if len(eligible_objects) > 1:
        groupdict = {k: list(g) for k, g in itertools.groupby(eligible_objects, group_funcs[prprty])}
        ambiguous_attrs = [[k, v] for k, v in groupdict.items() if len(v) > 1]
        if len(ambiguous_attrs) > 1:
          for ambiguous_attr in ambiguous_attrs:
            ambgs_obj_by_property[prprty]["attrs"][ambiguous_attr[0]] = ambiguous_attr[1]
        
    else:
      groupdict = {k: list(g) for k, g in itertools.groupby(objects, group_funcs[prprty])}
      ambiguous_attrs = [[k, v] for k, v in groupdict.items() if len(v) > 1]
      if len(ambiguous_attrs) > 1:
        ambgs_obj_by_property[prprty] = {"is_ambiguous": True, "attrs": {}}

        for ambiguous_attr in ambiguous_attrs:
          ambgs_obj_by_property[prprty]["attrs"][ambiguous_attr[0]] = ambiguous_attr[1]
      
      else:
        ambgs_obj_by_property[prprty] = {"is_ambiguous": False, "attrs": None}
  return ambgs_obj_by_property


def get_distance_from_coords(src_coords, trg_corrds, direction=None):
  # https://github.com/facebookresearch/clevr-dataset-gen/blob/f0ce2c81750bfae09b5bf94d009f42e055f2cb3a/image_generation/utils.py#L45
  distance = [(x - y)**2 for x, y in zip(src_coords, trg_corrds)]

  if direction is None:
    return np.sqrt(np.sum(distance))
  elif direction in ['left', 'right']:
    return np.sqrt(distance[0])
  elif direction in ['front', 'behind']:
    return np.sqrt(distance[2])

def collect_src_obj_dis_info(src_obj, trg_obj, all_src_objs, surroundings):
  # the relative information for the src obj correspondes to the trg obj

  dirc_reverse = {
      "left": "right",
      "right": "left",
      "front": "behind",
      "behind": "front"
  }

  # to record, number of closer src objects in the direction (more objects, more confusing)
  src_obj_info = {}

  for direc, surround_objects in surroundings.items():
    surround_object_ids = [x + 1 for x in surround_objects]
    in_direction = src_obj['idx'] in surround_object_ids

    direction = dirc_reverse[direc]

    distance = np.inf

    src_obj_info[direction] = {"in_direction": in_direction}

    if in_direction:
      # other ambiguous objects are not included
      eligible_src_obj = [x for x in all_src_objs if x['idx'] in surround_object_ids]

      if len(eligible_src_obj):
        all_src_obj_distances = [(x['idx'], get_distance_from_coords(x['pixel_coords'], trg_obj['pixel_coords'], direction)) for x in eligible_src_obj]

        all_src_obj_distances_sort = sorted(all_src_obj_distances, key=lambda x:x[1])
        num_other_src_obj_closer = [x[0] for x in all_src_obj_distances_sort].index(src_obj['idx'])

        src_obj_info[direction]['dis_pixel'] = all_src_obj_distances_sort[num_other_src_obj_closer][1]
        src_obj_info[direction]['num_closer_obj'] = num_other_src_obj_closer
      else:
        # no other objects are closer to the trg object
        dis_pixel = get_distance_from_coords(src_obj['pixel_coords'], trg_obj['pixel_coords'], direction)
        src_obj_info[direction]['dis_pixel'] = dis_pixel
        src_obj_info[direction]['num_closer_obj'] = 0
    else:
      src_obj_info[direction]['dis_pixel'] = distance

  # the rank of directions by closeness, starts from 1, more further, more visually distinguishable
  for dir_rank, dir_info in enumerate(sorted(src_obj_info.items(), key=lambda x:x[1]['dis_pixel'])):
    src_obj_info[dir_info[0]]['dir_rank'] = dir_rank + 1
  return src_obj_info


def generate_src_expr(src_obj, src_objs_by_dirc, 
                      num_src_attr=1, sort_dirc=False,
                      properties=["color", "material", "size"]):
  # num_attr_trg_obj
  # num_attr_src_obj

  exprs = []

  num_src_attr = min(num_src_attr, len(properties))

  for dirc, info in src_objs_by_dirc.items():
    if not info['in_direction']:
      continue

    template = "that is {} to the [SRC_OBJ].".format(dirc)
    relative_dis_template = "that is {} next to the [SRC_OBJ].".format(dirc)
    
    src_obj_expr = "[DIRECTION] of [SRC_EXPR]"
    relative_dis_obj_expr = "next to [SRC_EXPR]"

    src_obj_expr = src_obj_expr.replace("[DIRECTION]", dirc)

    # TODO: add additional properties
    for property_ in itertools.combinations(properties, num_src_attr):
      src_obj_attrs = [src_obj[x] for x in property_]
      src_obj_expr = "the {} {}".format(", ".join(src_obj_attrs), src_obj['shape'])
      src_expr = src_obj_expr.replace("[SRC_EXPR]", src_obj_expr)
      relative_dis_obj_src_expr = relative_dis_obj_expr.replace("[SRC_EXPR]", src_obj_expr)
      exprs.append({
          "expr": src_expr,
          "src_expr": src_obj_expr,
          "dirc": dirc,
          "dis_pixel_dirc": info['dis_pixel'],
          "rank_dirc": info['dir_rank'],
          "src_attr": [property_],
          "num_src_attr": num_src_attr,
          "relative_dis": False,
          "amb_trg_obj": False # whether the src object is ambiguous to the trg object
          })

      exprs.append({
          "expr": relative_dis_obj_src_expr,
          "src_expr": src_obj_expr,
          "dirc": dirc,
          "dis_pixel_dirc": info['dis_pixel'],
          "rank_dirc": info['dir_rank'],
          "src_attr": [property_],
          "num_src_attr": num_src_attr,
          "relative_dis": True,
          "amb_trg_obj": False # whether the src object is ambiguous to the trg object
          })
    
  return exprs

def distinguish_obj(obj_1_info, obj_2_info):
  # whether it is able to distinguish obj_1 and obj_2 by their expressions

  # identical expressions would be rejected (strict)
  if obj_1_info['expr'] == obj_2_info['expr']:
    return False
  
  obj_1_src_obj, obj_2_src_obj = obj_1_info['src_obj'], obj_2_info['src_obj']

  if len(obj_1_info['src_attr'][0]) * len(obj_2_info['src_attr'][0]) == 0:
    # one object has no additional information other than the shape
    return not obj_1_src_obj['shape'] == obj_2_src_obj['shape']

  # TODO: compatible with only one obj in obj_1_info and obj_2_info
  obj_attr_union = [x for x in ref_obj_1_expr['src_attr'][0] if x in ('color', 'size')]

  for obj_attr in obj_attr_union:
    if obj_1_src_obj[obj_attr] != obj_2_src_obj[obj_attr]:
      return True
  return False

def get_max_trg_covered_iou(trg_bbox, other_front_bboxes):
    max_iou = 0
    if not len(other_front_bboxes):
      return max_iou

    ious = bops.box_iou(torch.FloatTensor(other_front_bboxes), torch.FloatTensor(edge2bbox(target_object_bbox)).unsqueeze(0).repeat(len(other_front_bboxes), 1)).mean(1)
    max_iou = torch.max(ious).item()
    return max_iou

def left_and_right_between_indicator(trg_obj, ref_obj_1, ref_obj_2):
  if ref_obj_1['pixel_coords'][0] < ref_obj_2['pixel_coords'][0]:
    ref_obj_left, ref_obj_right = ref_obj_1, ref_obj_2
  else:
    ref_obj_left, ref_obj_right = ref_obj_2, ref_obj_1
  
  if trg_obj['pixel_coords'][0] > ref_obj_left['pixel_coords'][0] and trg_obj['pixel_coords'][0] < ref_obj_right['pixel_coords'][0]:
    return True
  return False

In [ ]:
# TODO: resolve the sphere, sphere

In [7]:
max_num_images = 300

queries_w_src_obj = []
queries_w_trg_obj_only = []

all_properties = ["color", "material", "size", "rotation"]

src_properties = ["color", "material", "size"]

dirc2expr = {
    "left": "left to",
    "right": "right to",
    "front": "in front of",
    "behind": "behind"
}

# direction_only

# order of src objects
# random: random order
# distance: from closet to furthest
# distance_reverse: from furthest to closet

# direction+distance: e.g. far behind/near behind

src_obj_order = "random"

# if True, place the direction with least distance to the last, otherwise random
sort_direction = False

# pixel means the pixel coordinates is used for calculate distance, o/w the 
# real 3d coordinates are used.
dist_coord = "pixel"

num_ref_attr_pattern = "(.*)num_ref_attr=([0-9]+)(.*)"

num_amb_src_object = 0

num_non_amb_src_object = 1

num_trg_attr = 1

# redundant information about the src object
# redundant objects
# redundant directions (is more than more direction satisfies)

trg_object_expr = None

num_avoid = 0

for scene_idx in range(max_num_images):

  scene_info = refexp_data_json['scenes'][scene_idx]
  img_name = scene_info['image_filename']

  assert img_name == scene_info['image_filename']

  try:
    objects_info = collect_ambiguous_objects(scene_info)
  except:
    continue

  ambiguous_properties = [(k, v['attrs']) for k, v in objects_info.items() if v['is_ambiguous']]
  break

  for ambiguous_property in ambiguous_properties:

    pprty = ambiguous_property[0]
    other_trg_pprties = [x for x in all_properties if x != pprty]

    for attr, attr_objs in ambiguous_property[1].items():
      # print(ambiguous_property[0], attr, [x['idx'] for x in attr_objs])
      for attr_obj_id in range(len(attr_objs)):
        target_object = attr_objs[attr_obj_id]
        target_object_id = target_object['idx']

        other_non_target_ambiguous_objs = [x for i, x in enumerate(attr_objs) if i != attr_obj_id]

        other_attrs = [target_object[x] for x in other_trg_pprties[:num_trg_attr-1]]
        trg_attr_mention = [attr] + other_attrs
        trg_object_expr = "The {} {}".format(", ".join(trg_attr_mention), target_object['shape'])

        target_object_bbox = scene_info['obj_bbox'][str(target_object_id)]

        other_bboxes = [v for k, v in scene_info['obj_bbox'].items() if k != str(target_object_id)]

        # bounding boxes that is in front of the target bbox
        other_bboxes_front = [edge2bbox(oth_bbox) for oth_bbox in other_bboxes if oth_bbox[1] < target_object_bbox[1]]
        max_iou = get_max_trg_covered_iou(target_object_bbox, other_bboxes_front)

        # information: trg obj, #attr used for the trg obj, #non-unique attr used for the trg obj, #src obj, #attr for each src obj

        non_target_ambiguous_objects = [x for i, x in enumerate(attr_objs) if i != attr_obj_id]

        left_objs = scene_info['relationships']['left'][target_object_id - 1]
        right_objs = scene_info['relationships']['right'][target_object_id - 1]
        front_objs = scene_info['relationships']['front'][target_object_id - 1]
        behind_objs = scene_info['relationships']['behind'][target_object_id - 1]

        surroundings = {
            "left": left_objs,
            "right": right_objs,
            "front": front_objs,
            "behind": behind_objs
        }

        expr_info_template = {
            "scene_id": scene_idx,
            "expr": None,
            "dirc": "self",
            'dis_pixel_dirc': None, 
            'rank_dirc': None, 
            'src_attr': [], 
            'num_src_attr': 0, 
            'num_trg_attr': num_trg_attr,
            'trg_attr': [ambiguous_property[0]] + other_trg_pprties[:num_trg_attr-1],
            'amb_trg_obj': False, 
            'trg_obj': target_object,
            'trg_obj_max_iou': max_iou,
            'image_filename': img_name
        }

        if len(left_objs) == 0:
          expr_info = expr_info_template.copy()
          expr_info['expr'] = trg_object_expr.replace("The", "The leftmost")
          expr_info['note'] = "leftmost"
          queries_w_trg_obj_only.append(expr_info)
        if len(right_objs) == 0:
          expr_info = expr_info_template.copy()
          expr_info['expr'] = trg_object_expr.replace("The", "The rightmost")
          expr_info['note'] = "rightmost"
          queries_w_trg_obj_only.append(expr_info)
        if len(front_objs) == 0:
          expr_info = expr_info_template.copy()
          expr_info['expr'] = trg_object_expr.replace("The", "The frontmost")
          expr_info['note'] = "frontmost"
          queries_w_trg_obj_only.append(expr_info)

          expr_info = expr_info_template.copy()
          expr_info['expr'] = trg_object_expr.replace("The", "The nearest")
          expr_info['note'] = "nearest"
          queries_w_trg_obj_only.append(expr_info)
        if len(behind_objs) == 0:
          expr_info = expr_info_template.copy()
          expr_info['expr'] = trg_object_expr.replace("The", "The behindmost")
          expr_info['note'] = "behindmost"
          queries_w_trg_obj_only.append(expr_info)

          expr_info = expr_info_template.copy()
          expr_info['expr'] = trg_object_expr.replace("The", "The furthest")
          expr_info['note'] = "furthest"
          queries_w_trg_obj_only.append(expr_info)

        if len(left_objs) * len(right_objs) * len(front_objs) * len(behind_objs):
          expr_info['expr'] = "{} in the middle.".format(trg_object_expr)
          expr_info['note'] = "middle"
          queries_w_trg_obj_only.append(expr_info)

        # other objects who do not share this attribute
        other_objects = list(itertools.chain.from_iterable([v for k, v in ambiguous_property[1].items() if k != attr]))
        other_object_ids = [x['idx'] for x in other_objects]

        if len(left_objs) * len(right_objs):
          ref_obj_1_infos, ref_obj_2_infos = {}, {}

          # the trg object is between a left object and a right object
          for num_src_attr in range(len(src_properties) + 1):
            for ref_obj_1_id in left_objs:
              ref_obj_1 = scene_info['objects'][ref_obj_1_id]

              if ref_obj_1['idx'] not in other_object_ids:
                continue

              ref_obj_1_dirs = collect_src_obj_dis_info(ref_obj_1, target_object, other_objects, surroundings)
              ref_obj_1_exprs = generate_src_expr(ref_obj_1, ref_obj_1_dirs, num_src_attr=num_src_attr,
                                            sort_dirc=sort_direction,
                                            properties=src_properties)
              
              if not len(ref_obj_1_exprs):
                continue

              for i in range(len(ref_obj_1_exprs)):
                ref_obj_1_exprs[i]['src_obj'] = ref_obj_1

              ref_obj_1_key = "ref_obj={}.num_ref_attr={}".format(ref_obj_1['idx'], num_src_attr)

              if ref_obj_1_key not in ref_obj_1_infos:
                ref_obj_1_infos[ref_obj_1_key] = []
              ref_obj_1_infos[ref_obj_1_key] += ref_obj_1_exprs

            for ref_obj_2_id in right_objs:
              ref_obj_2 = scene_info['objects'][ref_obj_2_id]

              if ref_obj_2['idx'] not in other_object_ids:
                continue

              ref_obj_2_dirs = collect_src_obj_dis_info(ref_obj_2, target_object, other_objects, surroundings)
              ref_obj_2_exprs = generate_src_expr(ref_obj_2, ref_obj_2_dirs, num_src_attr=num_src_attr,
                                            sort_dirc=sort_direction,
                                            properties=src_properties)
              
              if not len(ref_obj_2_exprs):
                continue

              for i in range(len(ref_obj_2_exprs)):
                ref_obj_2_exprs[i]['src_obj'] = ref_obj_2

              ref_obj_2_key = "ref_obj={}.num_ref_attr={}".format(ref_obj_2['idx'], num_src_attr)

              if ref_obj_2_key not in ref_obj_2_infos:
                ref_obj_2_infos[ref_obj_2_key] = []
              ref_obj_2_infos[ref_obj_2_key] += ref_obj_2_exprs
              
          all_between_exprs = set()

          for ref_obj_1_key in ref_obj_1_infos.keys():
            for ref_obj_2_key in ref_obj_2_infos.keys():
              ref_obj_1_num_attr = int(re.match(num_ref_attr_pattern, ref_obj_1_key).groups()[1])
              ref_obj_2_num_attr = int(re.match(num_ref_attr_pattern, ref_obj_2_key).groups()[1])

              if ref_obj_1_num_attr != ref_obj_2_num_attr:
                # the two referring objects use the same number of attributes
                continue

              ref_obj_1_exprs, ref_obj_2_exprs = ref_obj_1_infos[ref_obj_1_key], ref_obj_2_infos[ref_obj_2_key]  

              for ref_obj_1_expr in ref_obj_1_exprs:

                if ref_obj_1_expr['relative_dis']:
                  continue

                for ref_obj_2_expr in ref_obj_2_exprs:

                  if ref_obj_2_expr['relative_dis']:
                    continue

                  if not distinguish_obj(ref_obj_1_expr, ref_obj_2_expr):
                    # The exprs to ref_obj_1 and ref_obj_2 should be distinguishable
                    num_avoid += 1
                    continue

                  if any([left_and_right_between_indicator(trg_obj, ref_obj_1_expr['src_obj'], ref_obj_2_expr['src_obj']) for trg_obj in other_non_target_ambiguous_objs]):
                    # The other ambiguous objects does not meet the critieria
                    continue

                  between_expr_info = ref_obj_1_expr.copy()

                  between_expr = "{} that is between {} and {}.".format(trg_object_expr, ref_obj_1_expr['src_expr'], ref_obj_2_expr['src_expr'])

                  if between_expr in all_between_exprs:
                    continue

                  all_between_exprs.add(between_expr)
                  between_expr_info['scene_id'] = scene_idx
                  between_expr_info['src_obj'] = [ref_obj_1_expr['src_obj'], ref_obj_2_expr['src_obj']]
                  between_expr_info['trg_obj'] = target_object
                  between_expr_info['trg_obj_max_iou'] = max_iou
                  between_expr_info['trg_surrounding'] = {"left": len(left_objs), "right": len(right_objs), "front": len(front_objs), "behind": len(behind_objs)}
                  between_expr_info['trg_attr'] = [ambiguous_property[0]] + other_trg_pprties[:num_trg_attr-1]
                  between_expr_info["num_trg_att"] = len(src_expr['trg_attr'])

                  between_expr_info['expr'] = between_expr
                  between_expr_info['src_expr'] = [ref_obj_1_expr['src_expr'], ref_obj_2_expr['src_expr']]
                  between_expr_info['dirc'] = [ref_obj_1_expr['dirc'], ref_obj_2_expr['dirc']]
                  between_expr_info['dis_pixel_dirc'] = [ref_obj_1_expr['dis_pixel_dirc'], ref_obj_2_expr['dis_pixel_dirc']]
                  between_expr_info['rank_dirc'] = [ref_obj_1_expr['rank_dirc'], ref_obj_2_expr['rank_dirc']]
                  between_expr_info['note'] = "between.left-right.num_src_attr={}".format(ref_obj_1_num_attr)
                  between_expr_info['image_filename'] = img_name
                  queries_w_src_obj.append(between_expr_info)
                  

        for other_obj in other_objects:
          other_obj_dirs = collect_src_obj_dis_info(other_obj, target_object, other_objects, surroundings)

          for num_src_attr in range(len(src_properties)):
            # check uniqueness (on a different direction)
            src_exprs = generate_src_expr(other_obj, other_obj_dirs, num_src_attr=num_src_attr+1,
                                          sort_dirc=sort_direction,
                                          properties=src_properties)
            

            for src_expr in src_exprs:
              src_expr['scene_id'] = scene_idx
              src_expr['src_obj'] = other_obj
              src_expr['trg_obj'] = target_object
              src_expr['trg_obj_max_iou'] = max_iou
              src_expr['trg_surrounding'] = {"left": len(left_objs), "right": len(right_objs), "front": len(front_objs), "behind": len(behind_objs)}
              src_expr['trg_attr'] = [ambiguous_property[0]] + other_trg_pprties[:num_trg_attr-1]
              src_expr["num_trg_att"] = len(src_expr['trg_attr'])
              src_expr["expr"] = "{} that is {}.".format(trg_object_expr, src_expr['expr']) if src_expr['relative_dis'] else "{} that is {} {}.".format(trg_object_expr, dirc2expr[src_expr['dirc']], src_expr['expr'])
              src_expr["note"] = 'relative_dis' if src_expr['relative_dis'] else None
              src_expr['image_filename'] = img_name
              queries_w_src_obj.append(src_expr)
        # print(trg_object_expr, "scene_id={} #obj left={} right={} front={} behind={}\n".format(
        #     scene_idx, len(left_objs), len(right_objs), len(front_objs), len(behind_objs)
        # ))
  print("scene_id: {} trg_only: {} w_src_obj: {} num_avoid={}".format(scene_idx, len(queries_w_trg_obj_only), len(queries_w_src_obj), num_avoid))

In [31]:
[left_and_right_between_indicator(trg_obj, ref_obj_1_expr['src_obj'], ref_obj_2_expr['src_obj']) for trg_obj in other_non_target_ambiguous_objs]

[False, False, False, False, False]

In [ ]:
[(x['expr'], x['note'], x['trg_obj'], x["num_src_attr"]) for x in queries_w_src_obj[:10]]

In [34]:
expr_save_path = '/content/drive/MyDrive/Referal_expr'
expr_save_path = "/content"

version = 2

expr_save_path = "/content/drive/MyDrive/Profile/Projects/NLP/Proposition/data/expressions"

with open(join(expr_save_path, "expr.trg_obj_only.trg_attr={}.num_scene={}.v{}.json".format(num_trg_attr, max_num_images, version)), "w") as fp:
  for x in queries_w_trg_obj_only:
    fp.write(json.dumps(x) + "\n")
  fp.close()

with open(join(expr_save_path, "expr.src_trg_obj.trg_attr={}.num_scene={}.v{}.json".format(num_trg_attr, max_num_images, version)), "w") as fp:
  for x in queries_w_src_obj:
    fp.write(json.dumps(x) + "\n")
  fp.close()

In [ ]:
! zip -r /content/expr.zip /content/expr

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

pil_image = Image.open(BytesIO(imgdata)).convert("RGB")
image = np.array(pil_image)[:, :, [2, 1, 0]]
caption = 'The cyan sqaure.' # the caption can also be the simple concatonation of some random categories.
result, _ = glip_demo.run_on_web_image(image, caption, 0.5)
imshow(result, caption)

In [ ]:
!nvcc --version
!nvidia-smi

### Load CLEVR

In [ ]:
clevr_ref_expr_path = "/content/drive/MyDrive/Referal_expr/data/CLEVR"

In [ ]:
with open(join(clevr_ref_expr_path, 'clevr_ref+_train_refexps.json'), "r") as fp:
  ref_expr_data = json.load(fp)

In [ ]:
ref_expr_data.keys()

In [ ]:
ref_expr_data['refexps'][0]